Procol Buffers home page: https://developers.google.com/protocol-buffers/

Deffine protocol buffer file schema (Create cities.proto file).

1. Download precompiled binary version of the protocol buffer compiler (protoc) from https://github.com/google/protobuf/releases: "protoc-3.4.0-win32.zip" and add this location to your PATH environment variable

2. Install protobuf module (conda install protobuf).

3. Run protoc compiler on .proto file (protoc -I=source_dir --python_out=dest_dir source_dir/cities.proto).

In [1]:
import cities_pb2
import pandas as pd

In [2]:
# Load comma separated data containing highlevel info about main cities all over the World to pandas dataframe
# Data downloaded from from:https://simplemaps.com/data/world-cities
csv_file =pd.read_csv('simplemaps-worldcities-basic.csv', sep=',', header=0)

In [3]:
# Display few first rows of csv_file
csv_file.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan


In [4]:
# Open or create serialized file containing list of cities
serialized_file = open('Cities_proto', "wb")

In [5]:
# Create empty Cities object
Cities = cities_pb2.Cities()

In [6]:
#iterate through csv_file data frame and write data into Cities object
for i, row in csv_file.iterrows():
    new_city=Cities.City.add()
    new_city.name=csv_file.iloc[i]['city_ascii']
    new_city.lat=csv_file.iloc[i]['lat']
    new_city.lon=csv_file.iloc[i]['lng']
    new_city.country=csv_file.iloc[i]['country']
    new_city.population=int(csv_file.iloc[i]['pop'])
    iso = new_city.isocods.add()
    iso.code = str(csv_file.iloc[i]['iso2'])
    iso.type = cities_pb2.City_prop.ISO2
    iso = new_city.isocods.add()
    iso.code = str(csv_file.iloc[i]['iso3'])
    iso.type = cities_pb2.City_prop.ISO3

In [7]:
# Diplay 2 Cities from Cities object
Cities.City[0:2]

[name: "Qal eh-ye"
 lat: 34.98300013
 lon: 63.13329964
 country: "Afghanistan"
 population: 2997
 isocods {
   code: "AF"
   type: ISO2
 }
 isocods {
   code: "AFG"
   type: ISO3
 }, name: "Chaghcharan"
 lat: 34.5167011
 lon: 65.25000063
 country: "Afghanistan"
 population: 15000
 isocods {
   code: "AF"
   type: ISO2
 }
 isocods {
   code: "AFG"
   type: ISO3
 }]

In [8]:
#serialize message to string and write to serialized_file
serialized_file.write(Cities.SerializeToString())
serialized_file.close()

In [9]:
# Reading data from serialized_file
serialized_file= open('Cities_proto', "rb")
Cities_read = cities_pb2.Cities()
Cities_read.ParseFromString(serialized_file.read())


In [10]:
Cities_read.City[0:2]

[name: "Qal eh-ye"
 lat: 34.983001708984375
 lon: 63.13330078125
 country: "Afghanistan"
 population: 2997
 isocods {
   code: "AF"
   type: ISO2
 }
 isocods {
   code: "AFG"
   type: ISO3
 }, name: "Chaghcharan"
 lat: 34.516700744628906
 lon: 65.25
 country: "Afghanistan"
 population: 15000
 isocods {
   code: "AF"
   type: ISO2
 }
 isocods {
   code: "AFG"
   type: ISO3
 }]

In [25]:
# Function finding city and displaying it on the Folium map
import folium
%matplotlib inline

def find_city(country_name):
    map = folium.Map(tiles='Mapbox Bright')
    found=0
    for i in Cities_read.City:
        if i.name==country_name:
            folium.Marker([i.lat,i.lon], popup=i.name+\
                          "<br><i>Country: "+str(i.country)+"</i>"\
                          "<br><i>Population: "+str(i.population)+"</i>").add_to(map)
            map.location=[i.lat,i.lon]
            map.zoom_start=5
            found=1
    if found==0:
        print ('City not found in our DB')
    return map

In [26]:
#Searching for Krakow City
find_city('Krakow')

In [14]:
serialized_file.close()